In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!python -V

Python 3.6.9


In [2]:
!ls 'drive/My Drive'

◼NOT_ML


In [3]:
import tensorflow as tf
from tensorflow import keras

print(f'Tensorflow version: {tf.__version__}')
tf.python.client.device_lib.list_local_devices()
# print("# GPUs Available: ",
#     len(tf.config.experimental.list_physical_devices())
# )

Tensorflow version: 2.3.0


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 5259982231170734814, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 13097794076312885569
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 18303885401548394544
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14640891840
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 6244286574083197730
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"]

In [ ]:
# https://www.tensorflow.org/tutorials/images/classification

import time
import numpy as np
from tensorflow.keras.callbacks import TensorBoard


DATAPATH = "/content/drive/My Drive/datasets/eeveelutions/sap/Labeled/SoloEevee"
VAL_SPLIT = 0.15
IMG_HEIGHT = 160
IMG_WIDTH = 160
BATCH_SIZE = 32
EPOCHS = 10
RANDOM_SEED = 6110

timestamp = int(time.time())
tensorboard = TensorBoard(log_dir=f'tensorboard/eevee/{timestamp}')


FileNotFoundError: [Errno 2] No such file or directory: 'D:/User/long/WORKSPACE_ALT/__coding__/ML/Keras/Dataset/sap1231-251020/Labeled/SoloEevee'

In [ ]:
from tensorflow.keras.preprocessing import image_dataset_from_directory


train_ds = image_dataset_from_directory(
    DATAPATH,
    # label_mode='categorical',
    validation_split=VAL_SPLIT,
    subset="training",
    seed=RANDOM_SEED,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    interpolation="bilinear",
)
val_ds = image_dataset_from_directory(
    DATAPATH,
    validation_split=VAL_SPLIT,
    subset="validation",
    seed=RANDOM_SEED,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
)
print(train_ds.class_names)


# AUTOTUNE = tf.data.experimental.AUTOTUNE
# train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
# val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)


In [ ]:
from tensorflow.keras import layers


data_augmentation = keras.Sequential([
    layers.experimental.preprocessing.RandomFlip(
        "horizontal",
        input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)
    ),
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomContrast(0.1),
    layers.experimental.preprocessing.RandomTranslation(0.1, 0.1),
])

# https://keras.io/api/applications/ "Fine-tune InceptionV3 on a new set of classes"
# https://keras.io/api/applications/mobilenet/
model = keras.Sequential([
    # data_augmentation,
    layers.experimental.preprocessing.Rescaling(
        scale=1./127.5, offset=-1, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    keras.applications.mobilenet_v2.MobileNetV2(
        alpha=0.35, # width multiplier
        # depth_multiplier=1,
        include_top=False,
        pooling="avg",
        classes=9,
    ),
    # layers.GlobalAveragePooling2D(),
    layers.Dropout(0.1),
    layers.Dense(9,
        # activation='softmax'
    )
])

model.summary()


In [ ]:
# https://keras.io/api/models/model_training_apis/

model.compile(
    optimizer=keras.optimizers.SGD(lr=0.01, momentum=0.0),
    # loss='categorical_crossentropy',
    # optimizer=keras.optimizers.Adam(lr=0.002),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)


In [ ]:
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    verbose=1,
    callbacks=[tensorboard],
)

In [ ]:
import tensorflowjs as tfjs

model.save(f"EV-classify-{timestamp}.keras")
tfjs.converters.save_keras_model(model, "D:/User/long/WORKSPACE_ALT/__coding__/ML/Keras")